<a href="https://colab.research.google.com/github/Foruck/sentiment-analysis-demo/blob/attention/evaluate.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from google.colab import drive
drive.mount('/content/gdrive')

In [0]:
!apt-get install psmisc
!rm -rf log
!rm -rf bst_model
!mkdir log
!mkdir bst_model
!cp /content/gdrive/My\ Drive/en_5e-05_2018-12-23_04-48-53.pth en.pth

!ls

In [0]:
!pip uninstall torch
!pip uninstall torchvision
# http://pytorch.org/
from os.path import exists
from wheel.pep425tags import get_abbr_impl, get_impl_ver, get_abi_tag
platform = '{}{}-{}'.format(get_abbr_impl(), get_impl_ver(), get_abi_tag())
cuda_output = !ldconfig -p|grep cudart.so|sed -e 's/.*\.\([0-9]*\)\.\([0-9]*\)$/cu\1\2/'
accelerator = cuda_output[0] if exists('/dev/nvidia0') else 'cpu'

!pip install torch torchvision
import torch
print('Torch', torch.__version__, 'CUDA', torch.version.cuda)
print('Device:', torch.device('cuda:0'))

!pip install pytorch_pretrained_bert

In [0]:
import sys
import os
import datetime
import logging
import torch
import torch.nn as nn
import torch.nn.init as init
import torch.nn.functional as F
import torch.utils.data
import h5py
import numpy as np
import time
from torch.autograd import Variable
from pytorch_pretrained_bert import BertTokenizer, BertModel, BertForMaskedLM, BertAdam

In [0]:
LSTM_Hidden_Size = 256
embedding_length = 768
Sentence_Max_Length = 128

In [0]:
class BertTestSet(torch.utils.data.Dataset):
    def __init__(self, data, mask):
        self.data = data
        self.mask = mask
    
    def __len__(self):
        return self.data.shape[0]
    
    def __getitem__(self, idx):
        return self.data[idx, :], self.mask[idx, :]

class myBert(torch.nn.Module):
  def __init__(self, embedding_length=768, bert_path='bert-base-uncased', window=[7], classes=2, use_cuda=True):
    super(myBert, self).__init__()
    self.use_cuda = use_cuda
    self.num_filter = len(window)

    # Bert model
    self.bert = BertModel.from_pretrained(bert_path)

    self.dropout = nn.Dropout(p=0.5)

    # LSTM layers
    self.lstm1 = nn.LSTMCell(embedding_length, LSTM_Hidden_Size) # out n*1*LSTM_Hidden_Size
    self.lstm2 = nn.LSTMCell(embedding_length, LSTM_Hidden_Size)
    
    # Attention
    self.h1k1 = nn.Linear(LSTM_Hidden_Size, 64)
    self.k1e1 = nn.Linear(64, 1)
    self.h2k2 = nn.Linear(LSTM_Hidden_Size, 64)
    self.k2e2 = nn.Linear(64, 1)

    # FC layer
    self.fc = nn.Linear(LSTM_Hidden_Size * 2, classes)
    init.kaiming_normal_(self.fc.weight.data)
    self.fc.bias.data.fill_(0)
    
    self.simple_fc = nn.Linear(768, 2)
    
  def forward(self, inputs, mask):
    # Get Features
    inputs = self.bert(inputs, token_type_ids=None, attention_mask=mask, output_all_encoded_layers=False)[0]
    inputs = inputs.unsqueeze(1)
        
    # Go through Bi-LSTM
    n = inputs.shape[0]
    x0 = inputs.squeeze(1)
    if self.use_cuda:
      if not inputs.is_cuda:
        inputs = inputs.cuda()
        x0 = x0.cuda()
      cx1 = torch.zeros(n, LSTM_Hidden_Size).cuda()
      hx1 = torch.zeros(n, LSTM_Hidden_Size).cuda()
      hx2 = torch.zeros(n, LSTM_Hidden_Size).cuda()
      cx2 = torch.zeros(n, LSTM_Hidden_Size).cuda()
      hxs1 = torch.zeros((n, x0.shape[1], LSTM_Hidden_Size)).cuda()
      hxs2 = torch.zeros((n, x0.shape[1], LSTM_Hidden_Size)).cuda()
    else:
      if inputs.is_cuda:
        inputs = inputs.cpu()
        x0 = x0.cpu()
      cx1 = torch.zeros(n, LSTM_Hidden_Size)
      hx1 = torch.zeros(n, LSTM_Hidden_Size)
      cx2 = torch.zeros(n, LSTM_Hidden_Size)
      hx2 = torch.zeros(n, LSTM_Hidden_Size)
      hxs1 = torch.zeros((n, x0.shape[1], LSTM_Hidden_Size))
      hxs2 = torch.zeros((n, x0.shape[1], LSTM_Hidden_Size))
    
    for i in range(x0.shape[1]):
      hx1, cx1 = self.lstm1(x0[:, i, :], (hx1, cx1))
      hxs1[:, i, :] = hx1
      hx2, cx2 = self.lstm1(x0[:, x0.shape[1] - 1 - i, :], (hx2, cx2))
      hxs2[:, i, :] = hx2
      
    k1 = self.h1k1(hxs1) # n*128*64
    e1 = self.k1e1(k1).squeeze(2) # n*128*1 -> n*128
    e1 = F.softmax(e1, dim=1).unsqueeze(1) # n*1*128
    lstm1 = torch.matmul(e1, hxs1).squeeze(1)
    k2 = self.h2k2(hxs2) # n*128*64
    e2 = self.k2e2(k2).squeeze(2)
    e2 = F.softmax(e2, dim=1).unsqueeze(1)
    lstm2 = torch.matmul(e2, hxs2).squeeze(1)
    
    lstm_x = torch.cat((lstm1, lstm2), 1)
    
    x1 = self.dropout(self.fc(lstm_x))
    return x1

In [0]:
torch.set_default_tensor_type('torch.FloatTensor')
device = torch.device("cuda")
end_epoch = 60
tag = 'cn' # en英文，cn中文
checkpoint = ''
testfile = ''
bz = 16

In [0]:
if tag == 'cn':
  tokenizer = BertTokenizer.from_pretrained('bert-base-chinese')
  myModel = myBert(embedding_length, 'bert-base-chinese', use_cuda=True)
else:
  tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
  myModel = myBert(embedding_length, 'bert-base-uncased', use_cuda=True)
    
if args.checkpoint != '':
  state = torch.load(args.checkpoint)
  myModel.load_state_dict(state['model_state'])
    
myModel.to(device=device)
myModel = myModel.eval()

assert testfile != ''

In [0]:
xmltree = ET.parse(args.testfile)
xmlroot = xmltree.getroot()
lines = []
for review in xmlroot:
  s = review.text.strip()
  s = ' '.join(s.split())
  lines.append(s)
tot = len(lines)
test_index = torch.zeros((tot, 128)).long()
test_mask = torch.ones((tot, 128)).long()

In [0]:
for i in range(len(lines)):
  s = lines[i]
  tokenized_text = tokenizer.tokenize(s)[:128]
  indexed_tokens = tokenizer.convert_tokens_to_ids(tokenized_text)
  k = len(indexed_tokens)
  while k < 128:
    test_mask[i, k] = 0
    indexed_tokens.append(0)
    k += 1
  test_index[i, :] = torch.Tensor(indexed_tokens).long()
test_set = BertTestSet(test_index, test_mask)
test_loader = train_loader = torch.utils.data.DataLoader(dataset=test_set, batch_size=16, shuffle=False, num_workers=8)
test_res = torch.zeros(tot)

In [0]:
cnt = 0
for i, (index, mask) in enumerate(test_loader):
  k = index.shape[0]
  index = index.cuda()
  mask = mask.cuda()
  scores = myModel(index, mask)
  ans = torch.argmax(scores, dim=1).long()
  test_res[cnt:cnt + k] = ans
  cnt += k
    
cnt = 0
for review in xmlroot:
  if test_res[cnt] == 0:
    tmp = -1
  else:
    tmp = 1
  review.set('polarity','%d' % tmp)
  cnt += 1
xmltree.write('oup_' + args.testfile, encoding="UTF-8")